Before starting our work with the data, we needed to take an additional pre-processing step.

In the news articles from the 90s and early 2000s (up to and including 2001), the German umlauts (ö, ä, ü, ß, Ö, Ä, Ü) were often replaced with corresponding characters: 'oe', 'ae', 'ue', 'ss', 'OE', 'AE', and 'UE'. As a result, a word like 'Nürnberg' could be stored in the corpus as either 'Nuernberg' or 'N\xfcrnberg', depending on the publication date. To ensure that all (or nearly all) words have a consistent representation, we employ the `PyHunSpell` spellchecker.

In [1]:
# Import PyHunSpell
import hunspell

import pandas as pd 
import nltk
import re
import umlautf
import worker_umlaut
import multiprocessing as mp
import os

from datetime import datetime
from sacremoses import MosesDetokenizer # Detokenizing package
md = MosesDetokenizer() 

NUM_CORE = mp.cpu_count() # Set to the number of cores you want to use

C:\ProgramData\Anaconda3\envs\py27\lib\site-packages\sacremoses\truecase.py:26: UserWarning: You should really be using Python3!!! Tick tock, tick tock, https://pythonclock.org/
  "You should really be using Python3!!! "


In [2]:
# Make sure that you have downloaded nltk_data
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mokuneva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Correcting umlauts in Handelsblatt dataset

In [2]:
# Load the data
data_umlaut = pd.read_csv(os.path.join(os.getcwd().replace('Umlauts', 'Handelsblatt'), 'hb_umlauts_fix.csv'), encoding = 'utf-8', sep=';')

C:\ProgramData\Anaconda3\envs\py27\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Correct text 57565: some words and numbers are merged into a single token
tokens = nltk.word_tokenize(data_umlaut['texts'][57565])
tokens_new = []
# Split tokens containing text and numbers into two parts
# E.g., "OEsterreich1624,70" should be split into "OEsterreich" and "1624,70"
for t in tokens:
    match = re.match(r"([a-z]+)([0-9,]+)", t, re.I)
    if match:
        items = match.groups()
        tokens_new.append(items[0] + ' ' + items[1])
    else:
        tokens_new.append(t)
data_umlaut.loc[57565,'texts'] = md.detokenize(tokens_new)

C:\ProgramData\Anaconda3\envs\py27\lib\site-packages\sacremoses\truecase.py:26: UserWarning: You should really be using Python3!!! Tick tock, tick tock, https://pythonclock.org/
  "You should really be using Python3!!! "


In [5]:
# Use a spellchecker (umlautf.py function) to make umlaut replacements where necessary
startTime = datetime.now()

if __name__ == "__main__":
    list_of_texts = data_umlaut.texts
    list_of_objects = [umlautf.spell(i) for i in list_of_texts]
    
    pool = mp.Pool(NUM_CORE)
    corrected_umlauts = pool.map(worker_umlaut.worker_umlaut, ((obj) for obj in list_of_objects))
    pool.close()
    pool.join()
        
print datetime.now() - startTime

1:00:41.123000


In [6]:
data_umlaut['texts'] = corrected_umlauts 

In [7]:
data_umlaut.to_csv(os.path.join(os.getcwd().replace('Umlauts', 'Handelsblatt'), 'hb_umlauts_fixed.csv'), encoding='utf-8-sig', sep = ';')

# Correcting umlauts in dpa dataset

In [3]:
# Load the data
data_umlaut = pd.read_csv(os.path.join(os.getcwd().replace('Umlauts', 'dpa code'), 'dpa_umlauts_fix.csv'), encoding = 'utf-8', sep=';')

In [4]:
# Use a spellchecker (umlautf.py function) to make umlaut replacements where necessary
startTime = datetime.now()

if __name__ == "__main__":
    list_of_texts = data_umlaut.texts
    list_of_objects = [umlautf.spell(i) for i in list_of_texts]
    
    pool = mp.Pool(NUM_CORE)
    corrected_umlauts = pool.map(worker_umlaut.worker_umlaut, ((obj) for obj in list_of_objects))
    pool.close()
    pool.join()
        
print datetime.now() - startTime

0:08:29.532000


In [5]:
print(corrected_umlauts[0])

« Die Welt » (Bonn) zu golfkrise. In der Silvesternacht haben sich gute Freunde verabschiedet. Amerikaner. Ihr Kommando führt sie an den Golf. Sie gehen schweren Herzens. Gleichzeitig fiel in den ersten Minuten des neuen Jahres, in denen wir uns die Hände reichten und uns alles Gute wünschten, oft die Bemerkung: « bloß keinen Krieg. » der 15. Januar, an dem das Ultimatum endet, hat sich als Fixpunkt in unseren Köpfen verankert. Hoffnungen gelten jetzt den letzten diplomatischen Bemühungen, von denen man allerdings nicht weiß, ob sie nicht nur der Optik dienen. « bloß keinen Krieg » - das entspricht der Stimmungslage. Im Grunde genommen wünschen sich viele erneut einen Chamberlain, der, diesmal nicht aus München, sondern aus Bagdad kommend, ein Papier hochhält und « Frieden in unserer Zeit » verheißt. Wie lange hat dieser Frieden von 1938 gehalten? Saddam Hussein spricht heute davon, dass « Gott » auf seiner Seite sei. In den schrecklichen Zeiten, die nach München folgten, war von « Vor

In [6]:
data_umlaut['texts'].iloc[0]

u'\xab Die Welt \xbb (Bonn) zu golfkrise. In der Silvesternacht haben sich gute Freunde verabschiedet. Amerikaner. Ihr Kommando fuehrt sie an den Golf. Sie gehen schweren Herzens. Gleichzeitig fiel in den ersten Minuten des neuen Jahres, in denen wir uns die Haende reichten und uns alles Gute wuenschten, oft die Bemerkung: \xab bloss keinen Krieg. \xbb der 15. Januar, an dem das Ultimatum endet, hat sich als Fixpunkt in unseren Koepfen verankert. Hoffnungen gelten jetzt den letzten diplomatischen Bemuehungen, von denen man allerdings nicht weiss, ob sie nicht nur der Optik dienen. \xab bloss keinen Krieg \xbb - das entspricht der Stimmungslage. Im Grunde genommen wuenschen sich viele erneut einen Chamberlain, der, diesmal nicht aus Muenchen, sondern aus Bagdad kommend, ein Papier hochhaelt und \xab Frieden in unserer Zeit \xbb verheisst. Wie lange hat dieser Frieden von 1938 gehalten? Saddam Hussein spricht heute davon, dass \xab Gott \xbb auf seiner Seite sei. In den schrecklichen Zei

In [7]:
data_umlaut['texts'] = corrected_umlauts 

In [8]:
data_umlaut.to_csv(os.path.join(os.getcwd().replace('Umlauts', 'dpa code'), 'dpa_umlauts_fixed.csv'), encoding='utf-8-sig', sep = ';')